In [1]:
import pandas as pd
import numpy as np

In [2]:
cik_list = pd.read_excel("/Users/melodianking/Downloads/sentimentalanalysis_assignment/cik_list.xlsx")
max_row, max_col = cik_list.shape
print(max_row)

pd.set_option('display.max_colwidth',100) # to display full text in column
cik_list.head()

152


,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [3]:
cik_list.SECFNAME.head()


0    edgar/data/3662/0000950170-98-000413.txt
1    edgar/data/3662/0000950170-98-001001.txt
2    edgar/data/3662/0000950172-98-000783.txt
3    edgar/data/3662/0000950170-98-002145.txt
4    edgar/data/3662/0000950172-98-001203.txt
Name: SECFNAME, dtype: object

In [4]:
#adding the initial structure of the link in secfname
link = 'https://www.sec.gov/Archives/'
cik_list.SECFNAME = link+cik_list.SECFNAME
cik_list.SECFNAME.head()

0    https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt
1    https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt
2    https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt
3    https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt
4    https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt
Name: SECFNAME, dtype: object

In [5]:
import requests
import re, string, unicodedata
import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [6]:
#making the stopword set from basic english and the given list of stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopset = set(w.upper() for w in stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/melodianking/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/melodianking/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
import glob
path = "/Users/melodianking/Downloads/sentimentalanalysis_assignment/StopWords_Generic.txt"
glob.glob(path)
for filename in glob.glob(path):
    with open(filename, 'r') as f:
        text = f.read()
        text = re.sub(r"\s+\|\s+[\w]*" , "", text)        
        stopset.update(text.upper().split())
        #print(len(stopset))

In [8]:
from nltk.corpus import cmudict
nltk.download('cmudict')
d = cmudict.dict()

def syllables(word):
    #referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count+=1
    if count == 0:
        count +=1
    return count

def nsyl(word):
    try:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    except KeyError:
        #if word not found in cmudict
        return syllables(word)

[nltk_data] Downloading package cmudict to
[nltk_data]     /Users/melodianking/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [9]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_digits(text):
    return re.sub('[\d%/$]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_digits(text)
    return text

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_upper_case(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.upper()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words


def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopset:
            new_words.append(word)
    return new_words

# def stem_words(words):
#     """Stem words in list of tokenized words"""
#     stemmer = LancasterStemmer()
#     stems = []
#     for word in words:
#         stem = stemmer.stem(word)
#         stems.append(stem)
#     return stems

# def lemmatize_verbs(words):
#     """Lemmatize verbs in list of tokenized words"""
#     lemmatizer = WordNetLemmatizer()
#     lemmas = []
#     for word in words:
#         lemma = lemmatizer.lemmatize(word, pos='v')
#         lemmas.append(lemma)
#     return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_upper_case(words)
    words = remove_punctuation(words)
#     words = replace_numbers(words)
    words = remove_stopwords(words)
    return words


In [32]:

variables = ['positive_score','negative_score','polarity_score','average_sentence_length', 'percentage_of_complex_words',\
                   'fog_index','complex_word_count','word_count','uncertainty_score','constraining_score', 'positive_word_proportion',\
                   'negative_word_proportion', 'uncertainty_word_proportion', 'constraining_word_proportion' ]

In [33]:
import itertools

constraining_words_whole_report = pd.Series(name='constraining_words_whole_report')

df_col = [sec.lower() + '_' + var for sec,var in itertools.product(section_name,variables) ]
df = pd.DataFrame(columns=df_col)

<ipython-input-33-a5777f2468a2>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  constraining_words_whole_report = pd.Series(name='constraining_words_whole_report')


In [34]:
df.shape

(0, 42)

In [35]:
#usefull dictionaries
master_dict = pd.read_csv('/Users/melodianking/Downloads/sentimentalanalysis_assignment/LoughranMcDonald_MasterDictionary_2020.csv', index_col= 0)
constraining_dict = set(pd.read_excel('/Users/melodianking/Downloads/sentimentalanalysis_assignment/constraining_dictionary.xlsx',index_col = 0).index)
uncertainty_dict = set(pd.read_excel('/Users/melodianking/Downloads/sentimentalanalysis_assignment/uncertainty_dictionary.xlsx', index_col = 0).index)

In [36]:
cik_list.loc[64]

CIK                                                                          4962
CONAME                                                        AMERICAN EXPRESS CO
FYRMO                                                                      201407
FDATE                                                         2014-07-30 00:00:00
FORM                                                                         10-Q
SECFNAME    https://www.sec.gov/Archives/edgar/data/4962/0001193125-14-286961.txt
Name: 64, dtype: object

In [37]:
# saving all forms locally 
for i in range(max_row):
    text = requests.get(cik_list.SECFNAME[i]).text
    file_name = 'form' + str(i)
    f = open(file_name, 'a+')
    f.write(text)
    f.close()

In [38]:
for i in range(max_row):
    #print(i)
    file_name = '/Users/melodianking/Downloads/sentimentalanalysis_assignment/form' + str(i)
    text = open(file_name,'r').read()
    print('reading..',end = " ")
    
    #constraining_words_whole_report
    
#     constraining_words_whole_report_count = 0
#     for word in denoise_text(text).split():
#         if word in constraining_dict:
#             constraining_words_whole_report_count += 1
#     print('here...',end = "  ")
#     constraining_words_whole_report.loc[i] = constraining_words_whole_report_count
    
    ####################################
    df.loc[i] = np.zeros(42)
    # other variable per sections
    for j in range(3):
        if i in [63,64]:
            continue
        print(i,j,sep= '|',end = " ")
        exp = r".*(?P<start>ITEM [\d]\. " + re.escape(section[j]) + r")(?P<MDA>.*)(?P<body>[\s\S]*)(?P<end>ITEM \d|SIGNATURES)"
        regexp = re.compile(exp)
        s = regexp.search(text)
        
        if s:
            data = s.group('body')
            text = denoise_text(data)
            sent_list = sent_tokenize(text)
            sentence_length = len(sent_list)

            sample = text.split()
            sample = normalize(sample)
            word_count = len(sample)
            complex_word_count = 0
            
            for word in sample:
                if nsyl(word.lower()) > 2:
                    complex_word_count += 1
            
            average_sentence_length = word_count/sentence_length
            percentage_of_complex_words = complex_word_count/word_count
            fog_index = 0.4 * (average_sentence_length + percentage_of_complex_words)
            
            positive_score = 0
            negative_score = 0
            uncertainty_score = 0
            constraining_score = 0
            for word in sample:
                if word in master_dict.index:
                    #print("is here")
                    if master_dict.loc[word].Positive > 0:
                        #print("positive")
                        positive_score += 1
                    if master_dict.loc[word].Negative > 0:
                        negative_score += 1
                    if word in uncertainty_dict:
                        uncertainty_score += 1
                    if word in constraining_dict:
                        constraining_score += 1
            #print(positive_score)
            polarity_score = (positive_score-negative_score)/(positive_score + negative_score + .000001)
            positive_word_proportion = positive_score/word_count
            negative_word_proportion = negative_score/word_count
            uncertainty_word_proportion = uncertainty_score/word_count
            constraining_word_proportion = constraining_score/word_count
            
            df.loc[i][section_name[j].lower() + "_positive_score"] = positive_score
            df.loc[i][section_name[j].lower() + "_negative_score"] = negative_score
            df.loc[i][section_name[j].lower() + "_polarity_score"] = polarity_score
            df.loc[i][section_name[j].lower() + "_average_sentence_length"] = average_sentence_length
            df.loc[i][section_name[j].lower() + "_percentage_of_complex_words"] = percentage_of_complex_words
            df.loc[i][section_name[j].lower() + "_fog_index"] = fog_index
            df.loc[i][section_name[j].lower() + "_complex_word_count"] = complex_word_count
            df.loc[i][section_name[j].lower() + "_word_count"] = word_count
            df.loc[i][section_name[j].lower() + "_uncertainty_score"] = uncertainty_score
            df.loc[i][section_name[j].lower() + "_constraining_score"] = constraining_score
            df.loc[i][section_name[j].lower() + "_positive_word_proportion"] = positive_word_proportion
            df.loc[i][section_name[j].lower() + "_negative_word_proportion"] = negative_word_proportion
            df.loc[i][section_name[j].lower() + "_uncertainty_word_proportion"] = uncertainty_word_proportion
            df.loc[i][section_name[j].lower() + "_constraining_word_proportion"] = constraining_word_proportion 

reading.. 0|0 0|1 0|2 reading.. 1|0 1|1 1|2 reading.. 2|0 2|1 2|2 reading.. 3|0 3|1 3|2 reading.. 4|0 4|1 4|2 reading.. 5|0 5|1 5|2 reading.. 6|0 6|1 6|2 reading.. 7|0 7|1 7|2 reading.. 8|0 8|1 8|2 reading.. 9|0 9|1 9|2 reading.. 10|0 10|1 10|2 reading.. 11|0 11|1 11|2 reading.. 12|0 12|1 12|2 reading.. 13|0 13|1 13|2 reading.. 14|0 14|1 14|2 reading.. 15|0 15|1 15|2 reading.. 16|0 16|1 16|2 reading.. 17|0 17|1 17|2 reading.. 18|0 18|1 18|2 reading.. 19|0 19|1 19|2 reading.. 20|0 20|1 20|2 reading.. 21|0 21|1 21|2 reading.. 22|0 22|1 22|2 reading.. 23|0 23|1 23|2 reading.. 24|0 24|1 24|2 reading.. 25|0 25|1 25|2 reading.. 26|0 26|1 26|2 reading.. 27|0 27|1 27|2 reading.. 28|0 28|1 28|2 reading.. 29|0 29|1 29|2 reading.. 30|0 30|1 30|2 reading.. 31|0 31|1 31|2 reading.. 32|0 32|1 32|2 reading.. 33|0 33|1 33|2 reading.. 34|0 34|1 34|2 reading.. 35|0 35|1 35|2 reading.. 36|0 36|1 36|2 reading.. 37|0 37|1 37|2 reading.. 38|0 38|1 38|2 reading.. 39|0 39|1 39|2 reading.. 40|0 40|1 40|2 readi

In [39]:
for i in range(max_row):
    print(i,end = " ")
    file_name = '/Users/melodianking/Downloads/sentimentalanalysis_assignment/form' + str(i)
    text = open(file_name,'r').read()
    print('reading..',end = " ")
    
    #constraining_words_whole_report
    constraining_words_whole_report.loc[i] = 0
    constraining_words_whole_report_count = 0
    for word in denoise_text(text).split():
        if word in constraining_dict:
            constraining_words_whole_report_count += 1
    print('here...',end = "  ")
    constraining_words_whole_report.loc[i] = constraining_words_whole_report_count

0 reading.. here...  1 reading.. here...  2 reading.. here...  3 reading.. here...  4 reading.. here...  5 reading.. here...  6 reading.. here...  7 reading.. here...  8 reading.. here...  9 reading.. here...  10 reading.. here...  11 reading.. here...  12 reading.. here...  13 reading.. here...  14 reading.. here...  15 reading.. here...  16 reading.. here...  17 reading.. here...  18 reading.. here...  19 reading.. here...  20 reading.. here...  21 reading.. here...  22 reading.. here...  23 reading.. here...  24 reading.. here...  25 reading.. here...  26 reading.. here...  27 reading.. here...  28 reading.. here...  29 reading.. here...  30 reading.. here...  31 reading.. here...  32 reading.. here...  33 reading.. here...  34 reading.. here...  35 reading.. here...  36 reading.. here...  37 reading.. here...  38 reading.. here...  39 reading.. here...  40 reading.. here...  41 reading.. here...  42 reading.. here...  43 reading.. here...  44 reading.. here...  45 reading.. here...

In [42]:
df = pd.concat([cik_list,df,constraining_words_whole_report], axis = 1)
df.shape

(152, 56)

In [44]:
df.head(10)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,CIK,CONAME,FYRMO,FDATE,...,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt,3662,SUNBEAM CORP/FL/,199803,1998-03-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55,55
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt,3662,SUNBEAM CORP/FL/,199805,1998-05-15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt,3662,SUNBEAM CORP/FL/,199808,1998-08-13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt,3662,SUNBEAM CORP/FL/,199811,1998-11-12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt,3662,SUNBEAM CORP/FL/,199811,1998-11-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
5,3662,SUNBEAM CORP/FL/,199811,1998-11-25,10-Q/A,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002278.txt,3662,SUNBEAM CORP/FL/,199811,1998-11-25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
6,3662,SUNBEAM CORP/FL/,199812,1998-12-22,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002401.txt,3662,SUNBEAM CORP/FL/,199812,1998-12-22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16,16
7,3662,SUNBEAM CORP/FL/,199812,1998-12-22,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002402.txt,3662,SUNBEAM CORP/FL/,199812,1998-12-22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
8,3662,SUNBEAM CORP/FL/,199903,1999-03-31,NT 10-K,https://www.sec.gov/Archives/edgar/data/3662/0000950172-99-000362.txt,3662,SUNBEAM CORP/FL/,199903,1999-03-31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
9,3662,SUNBEAM CORP/FL/,199905,1999-05-11,10-K,https://www.sec.gov/Archives/edgar/data/3662/0000950170-99-000775.txt,3662,SUNBEAM CORP/FL/,199905,1999-05-11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,15


In [31]:
import pandas as pd
writer = pd.ExcelWriter('/Users/melodianking/Downloads/sentimentalanalysis_assignment/output.xlsx')
df.to_excel(writer)
writer.save()